In [2]:

# !pip install yfinance
import unicodedata
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import math


In [3]:
sharekhan_portfolio_df = pd.read_csv("sharekhan.csv",skiprows=4)
sharekhan_stocks_list=sharekhan_portfolio_df['Stock Name'].to_list()
start_date ="2023-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

In [4]:
icici_portfolio_df = pd.read_csv("icici_direct.csv")
icici_stocks_list = icici_portfolio_df['Stock Symbol'].to_list()
# icici_stocks_list
# len(icici_stocks_list)
# icici_portfolio_df.columns
# sharekhan_stocks_list

In [5]:

sharekhan_stocks_list = list(filter(lambda x: not isinstance(x, float) or not math.isnan(x),sharekhan_stocks_list))
# icici_stocks_list


In [6]:
df = pd.read_csv('EQUITY.csv')
# df=df[df[' ISIN NUMBER'].isin(icici_portfolio_df['ISIN Code'].to_list())]
stocks_list = df['SYMBOL'].tolist()
stocks_list_NSE = list(map(lambda x: x+".NS",stocks_list))
stocks_list_Sharekhan = list(map(lambda x: x+".NS",sharekhan_stocks_list))
final_list = [ls.replace(u'\xa0', u'')  for ls in stocks_list_Sharekhan]
final_list = final_list + stocks_list_NSE
final_list = list(dict.fromkeys(final_list))
# print(f" Final List = {stocks_list}")
# yahoo_data = yf.download(final_list,start=start_date,end=end_date)

In [7]:
len(stocks_list)
stocks_list_NSE=list(map(lambda x: x+".NS",stocks_list))
dfs=[]
def execute_in_batches(lst):
    batch_size = 100
    print(batch_size)
    for i in range(0, len(stocks_list), batch_size):
        yahoo_data = yf.download(lst[i:i+batch_size],start=start_date,end=end_date)
        dfs.append(yahoo_data)
    return pd.concat(dfs)
        
#         print("Batch", i // batch_size + 1, ":", lst[i:i+batch_size])
            
final_df = execute_in_batches(stocks_list_NSE)

100
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed

1 Failed download:
- GRASIM-RE.NS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed

1 Failed download:
- KALYANI.NS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[******

NameError: name 'yahoo_data' is not defined

In [9]:
# for s in final_list:
#     print(yahoo_data[('Adj Close',s)])


# yahoo_data.loc[:,('Volume','ABB.NS')]
# yahoo_data.to_csv("yahoo_data.csv")
yahoo_data=final_df
data_single_index=yahoo_data.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index(level=1)
# yahoo_data.stack(level=1).rename_axis(['Date', 'Ticker'])

In [10]:
import warnings
warnings.filterwarnings('ignore')

sell_df=pd.DataFrame()
buy_df = pd.DataFrame()

for ticker in stocks_list_NSE:
    try:
        
        temp_df = data_single_index[data_single_index['Ticker']==ticker]
        # temp_df['rolling_volume'] = temp_df['Volume'].rolling(window='30D').mean()
        temp_df.loc[:, 'rolling_volume'] = temp_df['Volume'].rolling(window='30D').mean()
        temp_df['rolling_volume']=temp_df['rolling_volume'].astype(int)
        temp_df['Volume']=temp_df['Volume'].astype(int)
        # temp_df['greater_than_rolling_mean'] = temp_df.groupby('Ticker')['Volume'].rolling(window=3).apply(lambda x: (x > x.mean()).all()).reset_index(0, drop=True)
        temp_df['vol_gt_avg']=temp_df.apply(lambda row:1 if row['Volume']> row['rolling_volume'] else 0 , axis=1)
        last_three_days = temp_df.iloc[-4:]
        price_increase_in_last_three_days = last_three_days['Close'].diff().dropna() > 0
        price_decrease_in_last_three_days = last_three_days['Close'].diff().dropna() < 0
        volume_flag = last_three_days['vol_gt_avg'].all()==1
    #     print("for ticker ", ticker)
    #     print(volume_flag)
        if price_decrease_in_last_three_days.all() and volume_flag:
    #         print(f' Sell Signal for : {ticker}')
    #         print(last_three_days)
            sell_df = pd.concat([sell_df,last_three_days])
        if price_increase_in_last_three_days.all() and volume_flag:
    #         print(f' Buy Signal for : {ticker}' )
    #         print(last_three_days)
            buy_df=pd.concat([buy_df,last_three_days])
    except:
        print("could not work for , ",ticker)
    

# print(f'BUY recommendation are {buy_df[["Ticker","Close"]]}' )
# print(f'SELL recommendation are {sell_df[["Ticker","Close"]]}')

could not work for ,  GRASIM-RE.NS
could not work for ,  KALYANI.NS
could not work for ,  NIRAJISPAT.NS


In [ ]:

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(buy_df[['Ticker','Close']])


In [ ]:
price_increase_in_last_three_days.all()

In [ ]:
last_three_days['vol_gt_avg'].all()

In [ ]:
buy_df['Ticker'].unique